In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Read the data
data = pd.read_csv('Introduction_Machine_Learning_Data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [4]:
#Gradient boosting is a method that goes through cycles to iteratively add models into an ensemble.

#Then, we start the cycle:

#1 First, we use the current ensemble to generate predictions for each observation in the dataset. 
#To make a prediction, we add the predictions from all models in the ensemble.

#2 These predictions are used to calculate a loss function (like mean squared error, for instance).

#3 Then, we use the loss function to fit a new model that will be added to the ensemble. 
#Specifically, we determine model parameters so that adding this new model to the ensemble will reduce the loss. 
#(Side note: The "gradient" in "gradient boosting" refers to the fact that we'll use gradient descent on the loss function 
#to determine the parameters in this new model.)

#4 Finally, we add the new model to ensemble, and ...

#5 ... repeat!

In [5]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

[20:43:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [6]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 264656.35086064064


In [7]:
#Parameter Tuning
#XGBoost has a few parameters that can dramatically affect accuracy and training speed. 
#The first parameters you should understand are:

#1 n_estimators

#n_estimators specifies how many times to go through the modeling cycle described above.
#It is equal to the number of models that we include in the ensemble.
#Too low a value causes underfitting
#Too high a value causes overfitting
#Typical values range from 100-1000

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[20:43:37] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 239315.69594302282


In [8]:
#2 early_stopping_rounds

#early_stopping_rounds offers a way to automatically find the ideal value for n_estimators
# It's smart to set a high value for n_estimators and then use early_stopping_rounds to find the optimal time to stop iterating.
#Setting early_stopping_rounds=5 is a reasonable choice. 
#In this case, we stop after 5 straight rounds of deteriorating validation scores.

my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[20:43:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 249591.1253267673


In [9]:
#3 learning_rate

#Instead of getting predictions by simply adding up the predictions from each component model, 
#we can multiply the predictions from each model by a small number (known as the learning rate) before adding them in.
#This means each tree we add to the ensemble helps us less. So, we can set a higher value for n_estimators without overfitting. 
#As default, XGBoost sets learning_rate=0.1

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[20:43:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 260962.86994661266


In [10]:
#4 n_jobs

#On larger datasets where runtime is a consideration, you can use parallelism to build your models faster. 
#It's common to set the parameter n_jobs equal to the number of cores on your machine.
#On smaller datasets, this won't help

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[20:43:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 260962.86994661266


In [11]:
##################################################

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('Intermediate_Machine_Learning_Train_Data.csv', index_col='Id')
X_test_full = pd.read_csv('Intermediate_Machine_Learning_Test_Data.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [13]:
# Define the model
my_model_1 = XGBRegressor(random_state=0) # Your code here

# Fit the model
my_model_1.fit(X_train,y_train) # Your code here

# Get predictions
predictions_1 = my_model_1.predict(X_valid) # Your code here

# Calculate MAE
mae_1 = mean_absolute_error(predictions_1,y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

[20:43:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 16803.434690710616


In [14]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_2.fit(X_train, y_train)

# Get predictions
predictions_2 = my_model_2.predict(X_valid)

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid)
print("Mean Absolute Error:" , mae_2)


[20:43:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 16084.123354559075


In [15]:
full_X_train = pd.concat([X_train, X_valid], axis=0)
full_y_train = pd.concat([y_train, y_valid], axis=0)

my_model_4 = XGBRegressor(n_estimators=1000, learning_rate=0.05)

# Fit the model
my_model_4.fit(full_X_train, full_y_train)

# Get predictions
predictions_4 = my_model_2.predict(X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': predictions_4})
output

[20:43:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Id,SalePrice
0,1461,117401.101562
1,1462,172459.250000
2,1463,190372.390625
3,1464,191226.937500
4,1465,183249.109375
...,...,...
1454,2915,75884.343750
1455,2916,80123.828125
1456,2917,168635.203125
1457,2918,115469.140625
